In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as la
# import scipy.linalg as la
import matplotlib as plot
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import altair as alt
import seaborn as sns

from sklearn.linear_model import LinearRegression as LR
import statsmodels.api as sm

import statsmodels.regression.linear_model as sm_lr
import statsmodels.tools.eval_measures as sm_eval
import sklearn.metrics as skm

C:\Users\grace\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\grace\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\grace\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Problem 1: cipher
Recall the substitution cipher from class:
'm it vbeh yjmbl... zftb.'
Decrypt it using an MCMC as described in class. Since the decryption process is non-deterministic, run your code ⇠7 times, each time for 4000 steps. At least a couple of these should result in a substitution key that is close enough to the correct one to complete the decryption task manually and find the true key and the original text. Note: the bigram count from the Google Corpus Data is available on Canvas.

In [70]:
def decode(msg,ky):
    return ''.join([ky[letter] if letter in alphabet else letter for letter in msg])

from collections import Counter
def log_f(bgrm_df,msg):
    msg = msg.lower()
    for mark in [",",".",";",""]:
        msg = msg.replace(mark,"")
    bigrams = Counter(map(''.join,zip(msg,msg[1:])))
    msg_df = pd.DataFrame()
    msg_df['bigram'] = bigrams.keys()
    msg_df['count'] = bigrams.values()
    msg_df = msg_df.set_index('bigram')
    log_f_score = 0
    
    in_df = set(msg_df[msg_df.columns[0]])
    for b in msg_df.index:
        try:
            log_f_score += np.log(bgrm_df.loc[b][0])*msg_df.loc[b][0]
        except:
            log_f_score += -7
    return log_f_score

In [3]:
message = 'm it vbeh yjmbl. qbl lgb tfwlgo iwn hbieo zftb, lgbh ziw lirbwflgmwx keft tb, lgbh ziw lirb wflgmwx tfeb. m it of iqfwb, iwnof umlgfjl gfdb lgil m ziw zfwkefwl lgbt umlgfjl kbie. lgb qmkb lgil giocfewb tb lgefjxg lgbob hbieo mo olmqq mw th giwno iwn th bhbo. ugblgbem givb ojcnjbn ml, m rwfu wfl. cjl of qfwx io ml mo lgbeb ml umqq obbrmlo fuw uih fjl, gbbnqboo fk lgb umqq lgil mo umlgmw tb. gb kbqqmw fzlfcbe wmwblbbw bmxglbbw fw i nih lgil uio of yjmbl iwn olmqqfw lgb ugfqb kefwl lgil lgb ieth ebdfel zfwkmwbn mlobqk lf lgb omwxqbobwlbwzb iqq yjmbl fw lgb ubolbew kefwl. gb gin kiqqbw kfeuien iwn qihfw lgb bielg io lgfjxg oqbbdmwx. ljewmwx gmt fvbe fwb oiu lgil gb zfjqnwfl givb ojkkbebn qfwx; gmo kizb gin iw badeboomfw fk ziqt, io lgfjxgiqtfol xqin lgb bwn gin zftb.'
bigram_df = pd.read_csv('bigrams.csv').set_index('bigram')
bigram_df[' count'] = bigram_df[' count'] + 0.00001
iterations = 4000
runs = 7

In [82]:
np.e**(np.log(8)-np.log(2))

3.9999999999999987

In [83]:
for r in range(runs):
    ### generate random key
    key = {}
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    alpha2 = alphabet
    for letter in alphabet:
        letter2 = alpha2[np.random.randint(0,len(alpha2))]
        key[letter] = letter2
        alpha2 = alpha2.replace(letter2,"")
    score = 0
    avg = [] #
    for n in range(iterations):
        ### swapping two random letters
        new_key = key.copy()
        a1, a2 = alphabet[np.random.randint(0,len(alphabet))], alphabet[np.random.randint(0,len(alphabet))]
        new_key[a1], new_key[a2] = key[a2], key[a1]

        ### scoring
        decoded, new_decoded = decode(message, key), decode(message, new_key)
        score, new_score = log_f(bigram_df, decoded), log_f(bigram_df, new_decoded)

        ## deciding to keep key
        if new_score > score:
            key = new_key
        else:
            p = np.e**(new_score - score)
            if np.random.rand() < p:
                key = new_key
        if n%500 == 0:
            print(score)
    print(score, key)
    print(decode(message, key))

8997.409883009463
10373.551572037984
10489.928675060508
10592.313815499781
10609.182851952603
10640.367041341307
10738.119387240959
10738.119387240959
10738.119387240959 {'a': 'x', 'b': 'e', 'c': 'b', 'd': 'p', 'e': 'r', 'f': 'o', 'g': 'h', 'h': 'y', 'i': 'a', 'j': 'u', 'k': 'f', 'l': 't', 'm': 'i', 'n': 'd', 'o': 's', 'p': 'z', 'q': 'l', 'r': 'k', 's': 'j', 't': 'm', 'u': 'w', 'v': 'v', 'w': 'n', 'x': 'g', 'y': 'q', 'z': 'c'}
i am very quiet. let the months and years come, they can takenothing from me, they can take nothing more. i am so alone, andso without hope that i can confront them without fear. the life that hasborne me through these years is still in my hands and my eyes. whetheri have subdued it, i know not. but so long as it is there it will seekits own way out, heedless of the will that is within me. he fellin october nineteen eighteen on a day that was so quiet and stillon the whole front that the army report confined itself to the singlesentence all quiet on the western f

In [ ]:
np.log(10)

# Problem 2: Knapsack
Solve (approximately) the knapsack problem with values =[99, 26, 71, 8, 28, 54, 5, 40, 100, 91, 27, 16, 78, 20, 97, 18, 63,
24, 1, 71], weights =[19, 17, 86, 52, 34, 58, 35, 96, 14, 92, 56, 51, 92, 70, 20, 83, 20, 18, 14, 25], max weight =200, =0.0005, and =0.0005 (i.e., increase by every 500 steps). The number of iterations per run should be 100,000 and since the solution is not deterministic run the code 10 times. Total values >400 are achievable — try to find such a choice.

In [106]:
values = [99,26,71,8,28,54,5,49,100,91,27,16,78,20,97,18,63,24,1,71]
weights = [19,17,86,52,34,58,35,96,14,92,56,51,92,70,20,83,20,18,14,25]
max_weight = 200
beta = 0.0005
delta = 0.0005
runs = 10
iters = 100000

In [107]:
items = pd.DataFrame()
items['values'], items['weights'] = values, weights

In [108]:
run_choices = []
run_value = []
for r in range(runs):
    weight = 201
    while weight > max_weight: ### generate valid random choice of items
        items['choice'] = np.random.randint(0,2,len(items))
        boltz = (np.e)**(beta*sum(items['choice']*items['values']))
        weight = sum(items['weights']*items['choice'])
    weight = 201
    for n in range(iters):
        new_choice = items['choice']
        while weight > max_weight: ### pick random index to toggle that is still valid
            i = np.random.randint(0,len(items)+1)
            new_choice[i] == (new_choice[i])*(-1)+1
            weight = sum(items['weights']*items['choice'])
        boltz_new = (np.e)**(beta*sum(items['choice']*items['values']))

        if boltz_new > boltz: ### accept / reject
            items['choice'] = new_choice
        else:
            if np.random.rand() > boltz_new / boltz:
                items['choice'] = new_choice
        if n%500==0:
            beta += delta
    run_choices.append(items['choice'])
    run_value.append(sum(items['values']*items['choice']))

In [109]:
max(run_value)

464

In [113]:
run_choices[np.argmax(run_value)].values

array([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1])

# Problem 3
Use the reference trigram count in American first names (available on Canvas) and similar logic to the substitution code decryption to implement an MCMC that generates 5or 6letter ’name-like’ strings.

In [87]:
name_bigrams = pd.read_csv('names_bigrams.csv').set_index('Unnamed: 0')

In [78]:
from collections import Counter
def log_f(bgrm_df,msg):
    msg = msg.lower()
    for mark in [",",".",";",""]:
        msg = msg.replace(mark,"")
    bigrams = Counter(map(''.join,zip(msg,msg[1:])))
    msg_df = pd.DataFrame()
    msg_df['bigram'] = bigrams.keys()
    msg_df['count'] = bigrams.values()
    msg_df = msg_df.set_index('bigram')
    log_f_score = 0
    
    in_df = set(msg_df[msg_df.columns[0]])
    for b in msg_df.index:
        try:
            log_f_score += np.log(bgrm_df.loc[b][0])*msg_df.loc[b][0]
        except:
            log_f_score += -7
    return log_f_score

In [88]:
iters = 1000
runs = 5
for r in range(runs):
    name = ''
    for i in range(np.random.randint(5,7)):
        name += chr(np.random.randint(97, 123))
    for n in range(iters):
        score = log_f(name_bigrams, name) ### score org name

        new_name = name ### make new name
        slice_here = np.random.randint(0,len(name))
        new_name = name[0:slice_here] \
            + chr(np.random.randint(97, 123)) \
            + name[1+slice_here:]

        new_score = log_f(name_bigrams, new_name) ### score new name
#         print(name, score)
#         print(new_name, new_score)
#         print("---")
        if new_score > score: ### compare scores
            name = new_name
        else:
            if np.random.rand() < new_score - score:
                name = new_name
    print("final:",name, score)

final: ananel 38.46736422364021
final: ellel 29.213595099287367
final: hanan 31.305993403776423
final: ariana 38.06453684430104
final: risha 29.983477269062018


# Problem 4
A mouse runs through a 5-room maze (below). Each time-step, the mouse moves to a di↵erent room. When in a room, it is equally likely to choose any of the doors leading out.

In [162]:
p = np.vstack([[0,1/3,1/4,1/3,0],
               [1/3,0,1/4,0,1/3],
               [1/3,1/3,0,1/3,1/3],
               [1/3,0,1/4,0,1/3],
               [0,1/3,1/4,1/3,0]])

array([[0.        , 0.33333333, 0.25      , 0.33333333, 0.        ],
       [0.33333333, 0.        , 0.25      , 0.        , 0.33333333],
       [0.33333333, 0.33333333, 0.        , 0.33333333, 0.33333333],
       [0.33333333, 0.        , 0.25      , 0.        , 0.33333333],
       [0.        , 0.33333333, 0.25      , 0.33333333, 0.        ]])

In [178]:
steady = la.eig(p)[1][:,0]
steady = steady / sum(steady)
steady

array([0.1875, 0.1875, 0.25  , 0.1875, 0.1875])

In [213]:
steps = 100000
position = np.random.randint(1,6)
record = {1:0,2:0,3:0,4:0,5:0}
at_3, at_5 = [],[]
for s in range(steps):
    record[position] += 1
    if position == 3:
        at_3.append(s)
    if position == 5:
        at_5.append(s)
    position = int(np.random.choice([1,2,3,4,5],1,p=p[:,position-1]))

In [214]:
record

{1: 18635, 2: 18845, 3: 25142, 4: 18653, 5: 18725}

In [220]:
back_3 = [at_3[x+1] - at_3[x] for x in range(len(at_3)-1)]
back_5 = [at_5[x+1] - at_5[x] for x in range(len(at_5)-1)]
np.mean(back_3), np.mean(back_5)

(3.977407422139135, 5.339831232642598)